In [1]:
%load_ext autoreload
%autoreload 2

import pyverilog
from pyverilog.vparser.parser import parse
from pyverilog.ast_code_generator.codegen import ASTCodeGenerator

import pandas as pd
import numpy as np
import utils

from tqdm import tqdm
tqdm.pandas(desc='Applying')

from pandarallel import pandarallel

## Pyverilog

In [2]:
filtered_deduplicated_index_df = utils.read_csv("data/verilog_partitions/filtered_deduplicated_file_index.csv")

In [3]:
index_partitions = np.array_split(filtered_deduplicated_index_df,25)

In [ ]:
pandarallel.initialize()
for i, idf in enumerate(index_partitions):
    if i > 18:
        print(f"Starting {i}")
        idf[['icarus_module_spans','icarus_exception']] = idf.parallel_apply(utils.pyverilog_parse_for_row, axis=1)
        idf.to_csv(f"data/icarus_partitions/icarus_processed_part_{i}.csv")

In [12]:
combined_df = utils.read_csv("data/icarus_partitions/icarus_processed_part_0.csv")
for i in range(1,25):
    partition_df = utils.read_csv(f"data/icarus_partitions/icarus_processed_part_{i}.csv")
    combined_df = pd.concat([combined_df,partition_df])
combined_df.to_csv(f"data/icarus_partitions/icarus_processed_full.csv")

In [2]:
combined_df = utils.read_csv("data/icarus_partitions/icarus_processed_full.csv")

In [3]:
pyverilog_modules_df = combined_df[combined_df.apply(lambda row: utils.list_is_not_empty(row,'icarus_module_spans'),axis=1)]
near_dedup_index_df = utils.read_csv("data/search_repo_indices/filtered_near_deduplicated_file_index.csv")
code_df = utils.read_csv("data/verilog_partitions/filtered_near_deduplicated_files_code.csv")

In [4]:
near_dedup_pyverilog_modules = pyverilog_modules_df[pyverilog_modules_df.index.isin(near_dedup_index_df.index)]
near_dedup_pyverilog_modules.index

Index(['8821', '310968', '234546', '88012', '114982', '190517', '194992',
       '54010', '274545', '307903',
       ...
       '314842', '314843', '314844', '314845', '314846', '314847', '314861',
       '314871', '314873', '314875'],
      dtype='object', length=54709)

In [ ]:
near_dedup_pyverilog_modules['code'] = code_df.loc[near_dedup_pyverilog_modules.index]['code']

In [ ]:
columns_to_keep = [c for c in near_dedup_pyverilog_modules.columns if not c in ['icarus_module_spans','icarus_exception','code','old_module_spans']]
only_modules_df = pd.DataFrame(columns=columns_to_keep + ['file_index','module_def','module_body'])
only_modules_df

In [7]:
near_dedup_pyverilog_modules['old_module_spans'] = near_dedup_pyverilog_modules['icarus_module_spans']

/tmp/ipykernel_22918/1164582715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  near_dedup_pyverilog_modules['old_module_spans'] = near_dedup_pyverilog_modules['icarus_module_spans']


In [ ]:
from pyverilog.vparser.parser import parse
row_directory = 'data/full_repos/permissive/166008742/src/led_driver_shift_reg.v'
# row_directory = "test4.v"
file_path = row_directory.rsplit("/",1)[0]
repo_path = "/".join(row_directory.split("/",4)[:4])
# ast, d = parse([row_directory], preprocess_include=[file_path, repo_path], preprocess_define=[])
# ast, d = parse([row_directory], preprocess_include=[], preprocess_define=[])

In [ ]:
from pyverilog.ast_code_generator.codegen import ASTCodeGenerator
codegen = ASTCodeGenerator()
rslt = codegen.visit(ast.description.children()[0])
print(rslt)

In [73]:
subset_df = near_dedup_pyverilog_modules.head(1000)

In [ ]:
pandarallel.initialize()
subset[['icarus_snippets','icarus_linenos','icarus_exception']] = subset_df.parallel_apply(utils.pyverilog_parse_for_row,axis=1)

In [ ]:
for file_index,row in tqdm(near_dedup_pyverilog_modules.iterrows()):
    modules = utils.get_pverilog_modules(row)
    code = row['code']
    module_span = row['icarus_module_spans']
    directory = row['directory']
    for module in modules:
        module_def, module_body = utils.split_pyverilog_module_def_and_body(module)
        only_modules_df.loc[len(only_modules_df)] = [row[ctk] for ctk in columns_to_keep] + [file_index,module_def,module_body]

## Verilator

In [2]:
index_df = utils.read_csv("data/search_repo_indices/filtered_near_deduplicated_file_index.csv")

In [3]:
index_partitions = np.array_split(index_df,25)

In [ ]:
pandarallel.initialize()
index_partitions[0][['verilator_xml_output_path','verilator_exception']] = index_partitions[0].parallel_apply(utils.verilator_parse_for_row,axis=1)
index_partitions[0].to_csv(f"data/verilator_partitions/icarus_processed_part_{0}.csv")

In [ ]:
# pandarallel.initialize()
for i, idf in enumerate(index_partitions):
    print(f"Starting {i}")
    # idf[['verilator_xml_output_path','verilator_exception']] = idf.parallel_apply(utils.verilator_parse_for_row,axis=1)
    idf[['verilator_xml_output_path','verilator_exception']] = idf.apply(utils.verilator_parse_for_row,axis=1)
    idf.to_csv(f"data/verilator_partitions/verilator_processed_part_{i}.csv")

In [7]:
combined_df = utils.read_csv("data/verilator_partitions/icarus_processed_part_0.csv")
combined_df.to_csv("data/verilator_partitions/verilator_processed_part_0.csv")
for i in range(1,25):
    partition_df = utils.read_csv(f"data/verilator_partitions/icarus_processed_part_{i}.csv")
    partition_df.to_csv(f"data/verilator_partitions/verilator_processed_part_{i}.csv")
    combined_df = pd.concat([combined_df,partition_df])
combined_df.to_csv(f"data/verilator_partitions/verilator_processed_full.csv")

## Combine code and index files together

In [4]:
verilator_df = utils.read_csv("data/verilator_partitions/verilator_processed_full.csv")
icarus_df = utils.read_csv("data/icarus_partitions/icarus_processed_full.csv")
nd_code_df = utils.read_csv("data/verilog_partitions/filtered_near_deduplicated_files_code.csv")

In [5]:
nd_verilator_df = verilator_df[verilator_df.index.isin(nd_code_df.index)]
nd_icarus_df = icarus_df[icarus_df.index.isin(nd_code_df.index)]

In [6]:
nd_verilator_df.columns

Index(['directory', 'repo_id', 'file_name', 'extension', 'no_lines',
       'max_line_len', 'generation_keywords', 'license_whitelist_keywords',
       'license_blacklist_keywords', 'icarus_module_spans', 'icarus_exception',
       'verilator_xml_output_path', 'verilator_exception'],
      dtype='object')

In [7]:
nd_icarus_df.columns

Index(['directory', 'repo_id', 'file_name', 'extension', 'no_lines',
       'max_line_len', 'generation_keywords', 'license_whitelist_keywords',
       'license_blacklist_keywords', 'icarus_module_spans',
       'icarus_exception'],
      dtype='object')

In [8]:
nd_code_df.columns

Index(['directory', 'repo_id', 'file_name', 'extension', 'code'], dtype='object')

In [ ]:
all_df = nd_icarus_df
all_df[['verilator_xml_output_path', 'verilator_exception']] = nd_verilator_df[['verilator_xml_output_path', 'verilator_exception']]
all_df['code'] = nd_code_df['code']

In [11]:
all_df.to_csv("data/full_unlabeled.csv")

Remove files with no code

In [2]:
all_df = utils.read_csv("data/full_unlabeled.csv")
print(len(all_df))

100606


In [3]:
def remove_all_comments(row):
    abstract_code, replaced = utils.abstract_strings(row['code'])
    no_comments_abstract_code = utils.remove_all_comments(abstract_code)
    return utils.recreate_string(no_comments_abstract_code,replaced)

In [4]:
all_df['code_no_comments'] = all_df.apply(remove_all_comments,axis=1)

In [5]:
all_with_code_df = all_df[all_df['code_no_comments'].str.len() > 0]
print(len(all_with_code_df)) 

100524


In [10]:
all_with_code_df.drop('code_no_comments',axis=1).to_csv("data/full_unlabeled.csv")